This notebook will demonstrate semantic understanding of texts using pre trained estimators and then use unsupervise learning to cluster and select some top recommendations

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Semantic Analysis

Importing pre trained embeddings

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")



Geberating embedding vector

In [ ]:
labels =  ["Where is my Phone","I Cant find my phone","My Phone is lost","This phone is made by apple",
           "This is a brand new phone","How much does this phone cost","What is the price of this phone",
          "Can you give me your phone","How Can I unlock this phone","Will this phone break","How durable is this phone"]
embeddings = embed(labels)

print(embeddings)

Grapically analysing the cosine similarity between sentences

In [ ]:
import seaborn as sns
import numpy as np

corr = tf.tensordot(embeddings,tf.transpose(embeddings), 1)

sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")

# Data Import and preprocessing

We will use the flipkart product catalog dataset to create a similarity matrix and using this matrix we will try to recommend products

In [ ]:
import pandas as pd
data = pd.read_csv('../input/flipkart-products/flipkart_com-ecommerce_sample.csv')

Some names are duplicate so we will need to only keep unique names

In [ ]:
name_list = data['product_name'].unique()

Here we gennerate the embeddings corresponding to the product names

In [ ]:
name_embeddings = embed(name_list) 

In [ ]:
name_embeddings.shape

Calculatinng the inner product of the two arrays to generate a similarity matrix of product names

In [ ]:
inner_prod = tf.tensordot(name_embeddings, tf.transpose(name_embeddings), 1)

Selecting a random product from product list

In [ ]:
import random
item = name_list[random.randint(0,len(name_list))]
item

finding the index of the product name

In [ ]:
item_index = np.where(name_list == item)[0][0]

Finding the 5 max values at row "item_index" of the product similarity matrix and then finding their corresponding indexes. Using these indexes finding the appropriate product names from the product list and displaying them.

In [ ]:
temp = inner_prod[item_index]
max_values = tf.math.top_k(temp,k = 6)

name_list[[max_values.indices]]

In [ ]:
class ContextRecomenderEngine:
    
    products = []
    embeddings = []
    path = ""
    embed = []
    inner_prod = []
    name_embeddings = []
    index = 0
    
    def __init__(self,embedding_tf_hub_path):
        self.path = embedding_tf_hub_path
    
    def fit(self,products):
        self.products = products
        self.embed = hub.load(self.path)
        self.name_embeddings = self.embed(products)
        self.inner_prod = tf.tensordot(name_embeddings, tf.transpose(name_embeddings), 1)
        print("Similarity Matrix size is {}".format(inner_prod.shape))
        
    def transform(self,item,top_k):
        self.index = np.where(self.products == item)[0][0]
        temp = self.inner_prod[self.index]
        max_values = tf.math.top_k(temp,k = top_k+1)
        return self.products[[max_values.indices]][1:]
        

In [ ]:
engine = ContextRecomenderEngine("https://tfhub.dev/google/universal-sentence-encoder-large/5")
engine.fit(name_list)

In [ ]:
item = name_list[random.randint(0,len(name_list))]
print(item)
engine.transform(item,50)